<a href="https://colab.research.google.com/github/DucBox/CyberSecurity/blob/main/Salsa20_%2B_DES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def rotl(value, shift, bits=32):
    return ((value << shift) | (value >> (bits - shift))) & ((1 << bits) - 1)

def quarter_round(a, b, c, d):
    a = (a + b) & 0xffffffff; d ^= a; d = rotl(d, 16)
    c = (c + d) & 0xffffffff; b ^= c; b = rotl(b, 12)
    a = (a + b) & 0xffffffff; d ^= a; d = rotl(d, 8)
    c = (c + d) & 0xffffffff; b ^= c; b = rotl(b, 7)
    return a, b, c, d

def salsa20_hash(state):
    x = list(state)
    for _ in range(10):  # 20 rounds, performed in pairs
        # Column rounds
        x[4], x[8], x[12], x[0] = quarter_round(x[4], x[8], x[12], x[0])
        x[9], x[13], x[1], x[5] = quarter_round(x[9], x[13], x[1], x[5])
        x[14], x[2], x[6], x[10] = quarter_round(x[14], x[2], x[6], x[10])
        x[3], x[7], x[11], x[15] = quarter_round(x[3], x[7], x[11], x[15])
        # Diagonal rounds
        x[1], x[2], x[3], x[0] = quarter_round(x[1], x[2], x[3], x[0])
        x[6], x[7], x[4], x[5] = quarter_round(x[6], x[7], x[4], x[5])
        x[11], x[8], x[9], x[10] = quarter_round(x[11], x[8], x[9], x[10])
        x[12], x[13], x[14], x[15] = quarter_round(x[12], x[13], x[14], x[15])
    return [(x[i] + state[i]) & 0xffffffff for i in range(16)]

def salsa20_key_schedule(key, nonce, counter):
    constants = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]
    key_schedule = constants[:1] + [int.from_bytes(key[i:i+4], 'little') for i in range(0, 16, 4)] \
                  + constants[1:3] + [counter & 0xffffffff, counter >> 32] \
                  + [int.from_bytes(nonce[i:i+4], 'little') for i in range(0, 8, 4)] + constants[3:]
    if len(key) == 32:
        key_schedule[1:5] += [int.from_bytes(key[i:i+4], 'little') for i in range(16, 32, 4)]
    elif len(key) != 16:
        raise ValueError("Key must be either 16 or 32 bytes long.")
    return key_schedule
def encrypt(plaintext, key, nonce):
    assert len(nonce) == 8, "Nonce must be 8 bytes long"
    encrypted = b''
    counter = 0
    while plaintext:
        block = plaintext[:64]
        plaintext = plaintext[64:]
        key_schedule = salsa20_key_schedule(key, nonce, counter)
        keystream = salsa20_hash(key_schedule)
        keystream_bytes = b''.join(word.to_bytes(4, 'little') for word in keystream)
        encrypted_block = bytes(a ^ b for a, b in zip(block, keystream_bytes[:len(block)]))
        encrypted += encrypted_block
        counter += 1
    return encrypted

def decrypt(ciphertext, key, nonce):
    return encrypt(ciphertext, key, nonce)

# Example usage
key = b"this_is_a_32_byte_key_for_salsa!"
nonce = b"nonce123"
plaintext = b"Hello, Salsa20!"

encrypted = encrypt(plaintext, key, nonce)
print(f"Encrypted: {encrypted.hex()}")

decrypted = decrypt(encrypted, key, nonce)
print(f"Decrypted: {decrypted.decode()}")



Encrypted: 0109fc750bc2b016264d845aedb744
Decrypted: Hello, Salsa20!


In [ ]:
def KSA(key):
    key_length = len(key)
    S = list(range(256))
    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % key_length]) % 256
        S[i], S[j] = S[j], S[i]
    return S

def PRGA(S):
    i = 0
    j = 0
    while True:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        K = S[(S[i] + S[j]) % 256]
        yield K

def RC4(key):
    S = KSA(key)
    return PRGA(S)


In [ ]:
def benchmark_stream_generation(cipher_func, key, nonce=None, data_size=1024*1024):
    start_time = time.time()
    # Generate a block of data
    data = b'\x00' * data_size
    if nonce:  # For Salsa20
        _ = cipher_func(data, key, nonce)
    else:  # For RC4, assuming cipher_func is adjusted to return bytes
        _ = cipher_func(data, key)
    end_time = time.time()
    return end_time - start_time

# Adjust RC4 to return bytes instead of a generator
def RC4_to_bytes(key, data):
    keystream = RC4(key)
    return bytes([next(keystream) for _ in range(len(data))])

# Benchmark Salsa20 and RC4
salsa20_time = benchmark_stream_generation(encrypt, key, nonce)
rc4_time = benchmark_stream_generation(RC4_to_bytes, key)

print(f"Salsa20 Generation Time: {salsa20_time} seconds")
print(f"RC4 Generation Time: {rc4_time} seconds")


Salsa20 Generation Time: 5.387058973312378 seconds
RC4 Generation Time: 0.0003523826599121094 seconds


In [ ]:
def key_sensitivity_test(encrypt_function, key, nonce=None):
    data = b'\x00' * 512  # Dùng một khối dữ liệu nhỏ để mã hóa
    flipped_key = bytearray(key)
    flipped_key[0] ^= 1  # Flip the first bit of the key

    encrypted1 = encrypt_function(data, key, nonce) if nonce else RC4_to_bytes(key, data)
    encrypted2 = encrypt_function(data, bytes(flipped_key), nonce) if nonce else RC4_to_bytes(bytes(flipped_key), data)

    # Calculate the number of differing bytes
    differences = sum(1 for x, y in zip(encrypted1, encrypted2) if x != y)
    return differences

# So sánh độ nhạy khóa
difference_salsa20 = key_sensitivity_test(encrypt, key, nonce)
difference_rc4 = key_sensitivity_test(RC4_to_bytes, key)

print(f"Salsa20 Key Sensitivity: {difference_salsa20} bytes differ")
print(f"RC4 Key Sensitivity: {difference_rc4} bytes differ")


Salsa20 Key Sensitivity: 511 bytes differ
RC4 Key Sensitivity: 510 bytes differ


In [ ]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00


In [ ]:
from Crypto.Cipher import DES, ARC4
from Crypto.Random import get_random_bytes
import time

# Padding để đảm bảo dữ liệu có độ dài là bội số của DES block size
def pad(text):
    while len(text) % 8 != 0:
        text += b' '
    return text

# Hàm mã hóa DES
def encrypt_des(plaintext, key):
    des = DES.new(key, DES.MODE_ECB)
    padded_text = pad(plaintext)
    encrypted_text = des.encrypt(padded_text)
    return encrypted_text

# Hàm giải mã DES
def decrypt_des(ciphertext, key):
    des = DES.new(key, DES.MODE_ECB)
    decrypted_text = des.decrypt(ciphertext)
    return decrypted_text.strip()

# Hàm mã hóa RC4
def encrypt_rc4(plaintext, key):
    rc4 = ARC4.new(key)
    encrypted_text = rc4.encrypt(plaintext)
    return encrypted_text

# Hàm giải mã RC4
def decrypt_rc4(ciphertext, key):
    rc4 = ARC4.new(key)
    decrypted_text = rc4.decrypt(ciphertext)
    return decrypted_text

# Đo lường thời gian
def measure_time(func, *args):
    start_time = time.time()
    func(*args)
    end_time = time.time()
    return end_time - start_time

# Tạo key ngẫu nhiên cho DES và RC4
# key_des = get_random_bytes(8)
key_rc4 = get_random_bytes (16)

key_des = b"mysecret"

# Plaintext mẫu
plaintext = b"Hello, DES!"

# Mã hóa DES
encrypted_des = encrypt_des(plaintext, key_des)
print("Encrypted Text (DES):", encrypted_des)
print("Encrypted Text (DES-hex):", encrypted_des.hex())

# Giải mã DES
decrypted_des = decrypt_des(encrypted_des, key_des)
print("Decrypted Text (DES):", decrypted_des.decode())


Encrypted Text (DES): b'\x90\xe3\x9d\x8de\xf1\xaa\xe6\xd5lo\xbb\xf7\xc68\xcf'
Encrypted Text (DES-hex): 90e39d8d65f1aae6d56c6fbbf7c638cf
Decrypted Text (DES): Hello, DES!


In [ ]:

# Tạo các case đa dạng để test
cases = [
    b"Short text.",
    b"https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4",
    b"https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4 https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4 It contains more characters and therefore, it is expected to take a longer time to encrypt and decrypt compared to shorter texts.",
    b"The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog"
]

# Đo lường thời gian mã hóa cho mỗi case
for i, data in enumerate(cases, 1):
    time_des = measure_time(encrypt_des, data, key_des)
    time_rc4 = measure_time(encrypt_rc4, data, key_rc4)
    print(f'Case {i}:')
    print(f'Time taken for DES encryption: {time_des:.6f} seconds')
    print(f'Time taken for RC4 encryption: {time_rc4:.6f} seconds\n')


Case 1:
Time taken for DES encryption: 0.001227 seconds
Time taken for RC4 encryption: 0.000046 seconds

Case 2:
Time taken for DES encryption: 0.000088 seconds
Time taken for RC4 encryption: 0.000043 seconds

Case 3:
Time taken for DES encryption: 0.000096 seconds
Time taken for RC4 encryption: 0.000045 seconds

Case 4:
Time taken for DES encryption: 0.000054 seconds
Time taken for RC4 encryption: 0.000035 seconds



In [ ]:
# Tạo các loại dữ liệu khác nhau để làm case test
binary_data = get_random_bytes(256)  # Dữ liệu nhị phân ngẫu nhiên
numeric_data = bytes([i for i in range(256)])  # Dãy số từ 0 đến 255
repeated_data = b"A" * 256  # Dữ liệu là ký tự lặp đi lặp lại
mixed_data = b"".join([bytes([i, 65]) for i in range(128)])  # Kết hợp giữa số và chữ

# Thêm các case dữ liệu mới vào danh sách
new_cases = [
    ("Binary Data", binary_data),
    ("Numeric Data", numeric_data),
    ("Repeated Character", repeated_data),
    ("Mixed Data", mixed_data),
]

# Đo lường thời gian mã hóa cho mỗi loại dữ liệu mới
for name, data in new_cases:
    time_des = measure_time(encrypt_des, data, key_des)
    time_rc4 = measure_time(encrypt_rc4, data, key_rc4)
    print(f'{name}:')
    print(f'Time taken for DES encryption: {time_des:.6f} seconds')
    print(f'Time taken for RC4 encryption: {time_rc4:.6f} seconds\n')


Binary Data:
Time taken for DES encryption: 0.000157 seconds
Time taken for RC4 encryption: 0.000043 seconds

Numeric Data:
Time taken for DES encryption: 0.000127 seconds
Time taken for RC4 encryption: 0.000034 seconds

Repeated Character:
Time taken for DES encryption: 0.000120 seconds
Time taken for RC4 encryption: 0.000036 seconds

Mixed Data:
Time taken for DES encryption: 0.000134 seconds
Time taken for RC4 encryption: 0.000036 seconds



In [ ]:
text_content = """dfiushfodsfhd9foadjfodsfaff"""
file_path = 'small_text.txt'

with open(file_path, 'w') as f:
    f.write(text_content)

In [ ]:
import os

file_path = 'random_file.bin'
file_size = 1024 * 1024  # 1MB

with open(file_path, 'wb') as f:
    f.write(os.urandom(file_size))

In [ ]:
import numpy as np
import time

IP = [58, 50, 42, 34, 26, 18, 10, 2,
      60, 52, 44, 36, 28, 20, 12, 4,
      62, 54, 46, 38, 30, 22, 14, 6,
      64, 56, 48, 40, 32, 24, 16, 8,
      57, 49, 41, 33, 25, 17, 9, 1,
      59, 51, 43, 35, 27, 19, 11, 3,
      61, 53, 45, 37, 29, 21, 13, 5,
      63, 55, 47, 39, 31, 23, 15, 7]

FP = [40, 8, 48, 16, 56, 24, 64, 32,
        39, 7, 47, 15, 55, 23, 63, 31,
        38, 6, 46, 14, 54, 22, 62, 30,
        37, 5, 45, 13, 53, 21, 61, 29,
        36, 4, 44, 12, 52, 20, 60, 28,
        35, 3, 43, 11, 51, 19, 59, 27,
        34, 2, 42, 10, 50, 18, 58, 26,
        33, 1, 41, 9, 49, 17, 57, 25]

EBox = [32,1,2,3,4,5,
            4,5,6,7,8,9,
            8,9,10,11,12,13,
            12,13,14,15,16,17,
            16,17,18,19,20,21,
            20,21,22,23,24,25,
            24,25,26,27,28,29,
            28,29,30,31,32,1]

SBox =[
		# S1
		[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7,
		 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8,
		 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0,
		 15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],

		# S2
		[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10,
		 3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5,
		 0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15,
		 13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],

		# S3
		[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8,
		 13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1,
		 13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7,
		 1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],

		# S4
		[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15,
		 13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9,
		 10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4,
		 3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],

		# S5
		[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9,
		 14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6,
		 4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14,
		 11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],

		# S6
		[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11,
		 10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8,
		 9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6,
		 4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],

		# S7
		[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1,
		 13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6,
		 1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2,
		 6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],

		# S8
		[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7,
		 1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2,
		 7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8,
		 2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
	]

F_PBox = [16, 7, 20, 21, 29, 12, 28, 17,
              1, 15, 23, 26, 5, 18, 31, 10,
              2, 8, 24, 14, 32, 27, 3, 9,
              19, 13, 30, 6, 22, 11, 4, 25 ]

key_PBox = [14,    17,   11,    24,     1,    5,
                  3,    28,   15,     6,    21,   10,
                 23,    19,   12,     4,    26,    8,
                 16,     7,   27,    20,    13,    2,
                 41,    52,   31,    37,    47,   55,
                 30,    40,   51,    45,    33,   48,
                 44,    49,   39,    56,    34,  53,
                 46,    42,   50,    36,    29,   32]


def xor(left,xorstream):
    xorresult = np.logical_xor(left,xorstream)

    xorresult  = xorresult.astype(int)

    return xorresult

def E_box(right):
    expanded = np.empty(48)
    j = 0
    for i in EBox:
        expanded[j] = right[i - 1]
        j += 1
    expanded = list(map(int,expanded))
    expanded = np.array(expanded)
    return expanded

def sboxloopup(sinput,x):
    tableno = x - 1
    row = int((np.array2string(sinput[0]) + np.array2string(sinput[5])),2)

    column = sinput[1:5]
    column = np.array2string(column)
    column = column[1:8].replace(" ", "")
    column = int(column,2)
    # print(column,"column")

    elementno = (16 * row) + column
    soutput = SBox[tableno][elementno]
    soutput = list(np.binary_repr(soutput, width=4))
    soutput= np.array(list(map(int, soutput)))
    return soutput

def sbox(sboxin):
#takes 48 bit input and return 32 bit
    sboxin1 = sboxin[0:6]
    sboxout1 = sboxloopup(sboxin1,1)
    sboxin2 = sboxin[6:12]
    sboxout2 = sboxloopup(sboxin2,2)
    sboxin3 = sboxin[12:18]
    sboxout3 = sboxloopup(sboxin3, 3)
    sboxin4 = sboxin[18:24]
    sboxout4 = sboxloopup(sboxin4, 4)
    sboxin5 = sboxin[24:30]
    sboxout5 = sboxloopup(sboxin5, 5)
    sboxin6 = sboxin[30:36]
    sboxout6 = sboxloopup(sboxin6, 6)
    sboxin7 = sboxin[36:42]
    sboxout7 = sboxloopup(sboxin7, 7)
    sboxin8 = sboxin[42:48]
    sboxout8 = sboxloopup(sboxin8, 8)
    sboxout = np.concatenate([sboxout1,sboxout2,sboxout3,sboxout4,sboxout5,sboxout6,sboxout7,sboxout8])
    return sboxout

def f_permute(topermute):
    permuted= np.empty(32)
    j = 0
    for i in F_PBox:
        permuted[j] = topermute[i - 1]
        j += 1
    return permuted

def f_function(right,rkey):
    expanded = E_box(right)
    xored = xor(expanded,rkey)
    sboxed = sbox(xored)
    xorstream = f_permute(sboxed)
    return xorstream

def round(data,rkey):
    l0 = data[0:32]
    r0 = data[32:64]
    xorstream = f_function(r0,rkey)
    r1 = xor(l0,xorstream)
    l1 = r0
    returndata = np.empty_like(data)
    returndata[0:32] = l1
    returndata[32:64] = r1
    return(returndata)

def permutation(data,x):
    #intial and final permutation conditional based on other passed value
    permute1 = np.empty_like(IP)
    if x == 0:
        j = 0
        for i in IP:
            permute1[j] = data[i-1]
            j += 1
        return(permute1)
    else:
        permute2 = np.empty_like(FP)
        k = 0
        for l in FP:
            permute2[k] = data[l-1]
            k += 1
        return(permute2)

def userinput():
    keyinp = input("Enter the key bits (56 bits) seperated by space " "").strip().split()
    datainp = input("Enter the data bits (64) to encrypt or decrypt seperated by space " "").strip().split()
    #change to 56 later
    lenofkey = 56
    #change to 64 later
    lenofdata = 64
    if len(datainp) == lenofdata and len(keyinp) == lenofkey:
        print("data entry accepted, data loaded succesfully")
        print("key entry accepted, key loaded succesfully")
    else:
        while len(datainp) != lenofdata:
            print("length of data entered ",len(datainp))
            datainp = input("Error in entered data. Enter the data (64 bits) to encrypt or decrypt seperated by space " "").strip().split()

        print("data entry accepted, data loaded succesfully")
        while len(keyinp) != lenofkey:
            print("length of key entered ", len(keyinp))
            keyinp = input("Error in entered key. Enter the key (56 bits) to encrypt or decrypt seperated by space " "").strip().split()
        print("key entry accepted, key loaded succesfully")
#also add functionality to accept 64 bit keys instead of 54
    return keyinp,datainp


def keyshift(toshift,n):
    if (n == 1) or (n == 2) or (n == 9) or (n == 16):
        toshift= np.roll(toshift,-1)
        return toshift
    else:
        toshift = np.roll(toshift, -2)
        return toshift

def keypermute(key16):
    keypermuted = np.empty([16,48])
    l = 0
    for k in key16:
        j = 0
        for i in key_PBox:
            keypermuted[l][j] = k[i - 1]
            j += 1
        l += 1
    return keypermuted

#
def keyschedule(key):
    left = key[0:28]
    right = key[28:56]
    shifted = np.zeros(56)
    key16 = np.zeros([16,56])
    for i in range(1,17):
        shifted[0:28] = keyshift(left,i)
        shifted[28:56] = keyshift(right,i)
        left = shifted[0:28]
        right = shifted[28:56]
#add shifted to key16 and return key16
        key16[i - 1] = shifted
#key16 is the final shifted 16 key pair now to permute
    key16 = keypermute(key16)
    key16 = [list(map(int, x)) for x in key16]
    key16 = np.array(key16)
    return key16


def main():
    key, data = userinput()
    operate = int(input("Choose 0 for encryption or Choose 1 for decryption "))
    starttime = time.time()
    key16 = keyschedule(key)

    if operate == 0:
        data = permutation(data,0)
# testing round function
        for i in range(16):
            data = round(data,key16[i])


#making left side right and right side left
        data = np.roll(data,32)
        data = (permutation(data, 1))
        print("Time taken to encrypt the data with DES is", time.time() - starttime)
        print("Encrypted data is", data)

    if operate == 1:
        data = permutation(data, 0)
        # testing round function now
        for i in range(16):
            data = round(data, key16[16 - (i + 1)])

        data = np.roll(data, 32)
        data = (permutation(data, 1))
        print("Time taken to decrypt the data with DES is", time.time() - starttime)
        print("Decrypted data is", data)


main()


Enter the key bits (56 bits) seperated by space mysecret
Enter the data bits (64) to encrypt or decrypt seperated by space Hello, DES!
length of data entered  2
Error in entered data. Enter the data (64 bits) to encrypt or decrypt seperated by space 12345678
length of data entered  1
Error in entered data. Enter the data (64 bits) to encrypt or decrypt seperated by space 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
length of data entered  16
Error in entered data. Enter the data (64 bits) to encrypt or decrypt seperated by space 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
data entry accepted, data loaded succesfully
length of key entered  1
Error in entered key. Enter the key (56 bits) to encrypt or decrypt seperated by space 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
length of key entered  15
Error in entered key. Enter the key (56 bits) to encrypt or decrypt seperated by space 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 